### Preprocessing 

In [1]:
import os
import xml.etree.ElementTree as ET
import cv2
import easyocr

reader = easyocr.Reader(['en'])

def parse_annotation(xml_file):
    """
    Parse the XML file to get the bounding box coordinates and the filename of the associated image.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    filename = root.find('filename').text
    for obj in root.iter('object'):
        xmlbox = obj.find('bndbox')
        x_min = int(xmlbox.find('xmin').text)
        y_min = int(xmlbox.find('ymin').text)
        x_max = int(xmlbox.find('xmax').text)
        y_max = int(xmlbox.find('ymax').text)
        return filename, (x_min, y_min, x_max, y_max)

def crop_and_resize_image(image_path, bbox, target_size=(224, 224)):
    """
    Crop the image to the bounding box and resize it to the target size.
    """
    image = cv2.imread(image_path)
    cropped_image = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]
    resized_image = cv2.resize(cropped_image, target_size)
    return resized_image

def preprocess_dataset(annotations_path, images_path, output_path, target_size=(224, 224)):
    """
    Process all images in the dataset, cropping to the bounding box and resizing to the target size.
    """
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for xml_file in os.listdir(annotations_path):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(annotations_path, xml_file)
            filename, bbox = parse_annotation(xml_path)
            image_path = os.path.join(images_path, filename)
            processed_image = crop_and_resize_image(image_path, bbox, target_size)
            
            # Save the processed image
            output_file_path = os.path.join(output_path, filename)
            cv2.imwrite(output_file_path, processed_image)

# Paths to your dataset
annotations_path = 'archive/annotations'
images_path = 'archive/images'
output_path = 'archive/processed_images'

# Preprocess the dataset
preprocess_dataset(annotations_path, images_path, output_path)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [3]:
from easyocr import Reader  # Assuming you are using EasyOCR for text detection
reader = Reader(['en'])  # Initialize the reader, specify the language as needed
import numpy as np

def load_images(image_directory, target_size=(224, 224)):
    images = []
    labels = []

    for filename in os.listdir(image_directory):
        if filename.endswith('.png'):
            img_path = os.path.join(image_directory, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, target_size)
            images.append(img)

            # Use OCR to read text from image
            results = reader.readtext(img)
            # You might want to refine how you extract the label from the results
            # Here we take the text with the highest confidence if there are multiple detections
            if results:
                # Sort results by confidence and take the text of the highest confidence result
                label = max(results, key=lambda result: result[2])[1]
                labels.append(label)
            else:
                labels.append('')  # or some placeholder text if no text detected

    return np.array(images), np.array(labels)




Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
image_directory = 'archive/processed_images'
img_height, img_width = 224, 224  # Should match your preprocessing size
batch_size = 32

images, labels = load_images(image_directory, (img_height, img_width))

['Wbh' '2' 'Hw' 'KA' '']


In [5]:
print(labels[:5])
print(images[:5])

['Wbh' '2' 'Hw' 'KA' '']
[[[[ 64  55  52]
   [ 52  43  40]
   [ 30  21  18]
   ...
   [250 246 247]
   [253 252 252]
   [255 255 255]]

  [[ 64  55  52]
   [ 52  43  40]
   [ 30  21  18]
   ...
   [250 246 247]
   [253 252 252]
   [255 255 255]]

  [[ 64  55  52]
   [ 52  43  40]
   [ 30  21  18]
   ...
   [250 246 247]
   [253 252 252]
   [255 255 255]]

  ...

  [[ 20  12  13]
   [ 33  25  26]
   [ 56  49  51]
   ...
   [154 154 154]
   [124 122 122]
   [107 105 105]]

  [[ 20  12  13]
   [ 33  25  27]
   [ 56  49  51]
   ...
   [154 154 154]
   [124 122 122]
   [107 105 105]]

  [[ 20  12  13]
   [ 33  25  26]
   [ 56  49  51]
   ...
   [154 154 154]
   [124 122 122]
   [107 105 105]]]


 [[[236 250 247]
   [236 249 247]
   [234 247 244]
   ...
   [119 122 126]
   [128 130 134]
   [129 132 135]]

  [[236 250 247]
   [236 250 246]
   [234 247 244]
   ...
   [119 122 125]
   [128 131 134]
   [129 132 135]]

  [[236 250 247]
   [236 250 246]
   [234 247 244]
   ...
   [119 122 125]
   

In [7]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [2]:
# import tensorflow as tf
# # from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import train_test_split
# import numpy as np
# import cv2
# import os

# # Parameters
# image_directory = 'archive/processed_images'
# img_height, img_width = 224, 224  # Should match your preprocessing size
# batch_size = 32

# # Load and preprocess images
# def load_images(image_directory, target_size=(224, 224)):
#     images = []
#     labels = []  # Assuming labels can be derived from file names or another source

#     for filename in os.listdir(image_directory):
#         if filename.endswith('.png'):
#             img_path = os.path.join(image_directory, filename)
#             img = cv2.imread(img_path)
#             img = cv2.resize(img, target_size)
#             images.append(img)
#             # Extract label from filename or another source
#             label = reader.readtext(img)
#             # label = filename.split('_')[0]  # Modify as per your naming convention
#             labels.append(label)

#     return np.array(images), np.array(labels)

# images, labels = load_images(image_directory, (img_height, img_width))

# # Split the dataset


2024-04-07 22:24:15.515644: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (433,) + inhomogeneous part.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(labels)), activation='softmax')  # Assuming a classification task
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [9]:
# You may need to convert labels to numeric if they are not already
# This is a simplistic example; in practice, you would use label encoding or one-hot encoding

# Fit the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=batch_size)


Epoch 1/10


2024-04-07 22:43:19.899808: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'Cast_25' defined at (most recent call last):
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/kreenatotala/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kreenatotala/Library/Python/3.9/lib/python/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/kreenatotala/Library/Python/3.9/lib/python/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/kreenatotala/Library/Python/3.9/lib/python/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asy